# Part 1: Cross validation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [2]:
import warnings
warnings.filterwarnings('ignore')

## 1a) Validation set approach
Objective: Use validation set approach to evaluate performance of model predicting `mpg` in `Auto` dataset based on predictor `horsepower`.

In [3]:
Auto =  load_data('Auto')

In [4]:
train, validation = train_test_split(Auto, test_size=0.3)

In [5]:
train

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
380,25.0,6,181.0,110,2945,16.4,82,1,buick century limited
120,15.0,8,318.0,150,3399,11.0,73,1,dodge dart custom
80,28.0,4,97.0,92,2288,17.0,72,3,datsun 510 (sw)
55,26.0,4,91.0,70,1955,20.5,71,1,plymouth cricket
245,39.4,4,85.0,70,2070,18.6,78,3,datsun b210 gx
...,...,...,...,...,...,...,...,...,...
360,20.2,6,200.0,88,3060,17.1,81,1,ford granada gl
341,35.1,4,81.0,60,1760,16.1,81,3,honda civic 1300
382,26.0,4,156.0,92,2585,14.5,82,1,chrysler lebaron medallion
302,37.3,4,91.0,69,2130,14.7,79,2,fiat strada custom


In [6]:
validation

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
10,15.0,8,383.0,170,3563,10.0,70,1,dodge challenger se
128,26.0,4,122.0,80,2451,16.5,74,1,ford pinto
189,22.0,6,225.0,100,3233,15.4,76,1,plymouth valiant
20,25.0,4,110.0,87,2672,17.5,70,2,peugeot 504
...,...,...,...,...,...,...,...,...,...
14,24.0,4,113.0,95,2372,15.0,70,3,toyota corona mark ii
176,23.0,4,120.0,88,2957,17.0,75,2,peugeot 504
292,34.1,4,86.0,65,1975,15.2,79,3,maxda glc deluxe
185,17.5,8,305.0,140,4215,13.0,76,1,chevrolet chevelle malibu classic


In [7]:
design = MS(['horsepower']).fit(train)
X_train = design.transform(train)
X_train

,intercept,horsepower
380,1.0,110
120,1.0,150
80,1.0,92
55,1.0,70
245,1.0,70
...,...,...
360,1.0,88
341,1.0,60
382,1.0,92
302,1.0,69


In [8]:
y_train = train.mpg

In [9]:
model = sm.OLS(y_train,X_train)
results = model.fit()
summarize(results)

,coef,std err,t,P>|t|
intercept,40.9083,0.901,45.399,0.0
horsepower,-0.1672,0.008,-20.175,0.0


In [10]:
y_valid_actual = validation.mpg
X_validation = design.transform(validation)
y_valid_predicted = results.predict(X_validation)
MSE = np.mean((y_valid_actual - y_valid_predicted)**2)
MSE

24.85863566427132

In [11]:
X_validation

,intercept,horsepower
3,1.0,150
10,1.0,170
128,1.0,80
189,1.0,100
20,1.0,87
...,...,...
14,1.0,95
176,1.0,88
292,1.0,65
185,1.0,140


In [12]:
def evalMSE(predictors,
           train,
           validation):
    # build design matrix and response vector
    design = MS(predictors).fit(train)
    X_train = design.transform(train)
    y_train = train.mpg 

    # train model
    model = sm.OLS(y_train,X_train)
    results = model.fit()

    # compute MSE
    y_valid_actual = validation.mpg
    X_validation = design.transform(validation)
    y_valid_predicted = results.predict(X_validation)
    MSE = np.mean((y_valid_actual - y_valid_predicted)**2)
    return MSE

In [13]:
evalMSE(['horsepower'], train, validation)

24.85863566427132

Let's compute the MSE for linear regression models including successively higher polynomial terms of `horsepower`.

In [14]:
predictors = [poly('horsepower', 2)] # choose powers of horsepower as predictors up to degree i+1
MS(predictors).fit_transform(Auto)

,intercept,"poly(horsepower, degree=2)[0]","poly(horsepower, degree=2)[1]"
0,1.0,0.033544,-0.052723
1,1.0,0.079529,-0.009329
2,1.0,0.059821,-0.036300
3,1.0,0.059821,-0.036300
4,1.0,0.046682,-0.047302
...,...,...,...
387,1.0,-0.024266,-0.010260
388,1.0,-0.068938,0.096569
389,1.0,-0.026894,-0.005763
390,1.0,-0.033463,0.006459


In [15]:
MSE

24.85863566427132

In [16]:
from ISLP.models import poly
MSE = []
for i in range(5):
    predictors = [poly('horsepower', i+1)] # choose powers of horsepower as predictors up to degree i+1
    err = evalMSE(predictors, train, validation)
    MSE.append(err)

In [17]:
MSE

[24.858635664271315,
 20.419948439347653,
 20.952145200197954,
 20.830573969103334,
 19.642665467723297]

## 1b) Cross validation

Cross validation is implemented most comfortably in `scikit-learn`. In order to use the `scikit-learn` implementation of cross validation with our `statsmodels` linear model, we use the wrapper `sklearn_sm` provided by the `ISLP` library. From the lab in Chapter 5:

"The class `sklearn_sm()` has as its first argument a model from `statsmodels`. It can take two additional optional arguments: `model_str` which can be used to specify a formula, and `model_args` which should be a dictionary of additional arguments used when fitting the model. For example, to fit a logistic regression model we have to specify a family argument. This is passed as `model_args={'family':sm.families.Binomial()}`."

After specifying our design matrix `X` and the vector `y` we call the `scikit-learn` function `cross_validate` (see [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)).

The result is a dictionary which among others contains the `test_score` which we are interested when we use cross validation to estimate the test error.

### Example: LOOCV for a simple linear regression model

In [18]:
model = sklearn_sm(sm.OLS,
                  MS(['horsepower']))
X = Auto.drop(columns = ['mpg'])
y = Auto['mpg']

The function `cross_validate` is the original `scikit-learn` function which carries out cross validation. We provide the following arguments:
- a model which needs `fit()` and `predict()` methods
- a design matrix `X` and a vector of training labels `y`
- the parameter `cv` specifying the number of folds for cross validation.

See the [official documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) for more details.

In [19]:
cv_results = cross_validate(model,
                           X,
                           y,
                           cv = Auto.shape[0])

In [20]:
cv_results['test_score']

array([2.02001002e+00, 1.25092412e+00, 3.06805164e+00, 6.79901984e-02,
       7.08255629e-01, 4.13566745e+01, 8.13755358e+01, 6.71494767e+01,
       9.70498847e+01, 2.63430368e+01, 3.67428697e+00, 4.70741691e-01,
       1.60507789e+00, 9.70498847e+01, 8.89557151e-01, 8.69418110e+00,
       4.41228965e+01, 3.06562225e+01, 9.16549732e-01, 4.53185378e+01,
       1.45705281e+00, 3.00983554e+00, 3.54617605e-03, 1.52964088e+01,
       2.25022208e+01, 1.67905446e+01, 2.76735351e+00, 1.85354648e+01,
       2.29957474e-01, 9.16549732e-01, 5.18379999e+00, 3.54617605e-03,
       2.66745510e+01, 5.44791120e+01, 5.14078324e+01, 4.99405256e+01,
       3.80360006e+01, 1.19884585e-02, 2.91033003e+00, 3.23104363e+00,
       5.16691118e+00, 2.32487332e-01, 1.06678908e-02, 4.82615264e-01,
       2.10211114e+01, 4.35585197e+01, 2.66745510e+01, 6.51231162e+01,
       1.13690418e+01, 5.18379999e+00, 1.25085727e+00, 4.27873211e+00,
       1.77161819e+00, 3.58044876e+01, 1.21519855e+01, 8.41044041e+00,
      

In [21]:
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792922

cv_results

### b) LOOCV to compare different models

We can repeat this procedure to compare different models. In the following we do this with various polynomial fits:

In [22]:
cv_error = np.zeros(5) #initialize empty vector for CV errors of 5 models
M = sklearn_sm(sm.OLS)
y = Auto['mpg']

# loop over 5 different polynomial models
for i in range(5):
    # choose power of horsepowers as predictors up to degree i+1
    predictors = [poly('horsepower', i+1)]
    X = MS(predictors).fit_transform(Auto)
    #perform cross validation for polynomial model of degree i+i
    cv_results = cross_validate(M,
                               X,
                               y,
                               cv = Auto.shape[0])
    # save cross validation error into result vector
    cv_error[i] = np.mean(cv_results['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443031, 19.03321385])

### c) General cross validation

Instead of using $K = n$ folds such as above (resulting in Leave-One-Out-Cross-Validation (LOOCV)) we can also specify a smaller integer $K$ of folds. There are two possibilities for this:
- set `cv = K`,
- specify a cross validation generator such as `KFold` (see [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html))

Below we use the second approach, which is generally preferred as we do better control the kind of split when using `KFold`.

In [23]:
cv_error = np.zeros(5)

cross_val = KFold(n_splits = 10,
                 shuffle = True,
                 random_state = 42)

M = sklearn_sm(sm.OLS)
y = Auto['mpg']

# loop over 5 different polynomial models
for i in range(5):
    # choose power of horsepowers as predictors up to degree i+1
    predictors = [poly('horsepower', i+1)]
    X = MS(predictors).fit_transform(Auto)
    #perform cross validation for polynomial model of degree i+i
    cv_results = cross_validate(M,
                               X,
                               y,
                               cv = cross_val)
    # save cross validation error into result vector
    cv_error[i] = np.mean(cv_results['test_score'])
cv_error

array([24.1998082 , 19.22863661, 19.26626535, 19.35109227, 19.02323325])

# Part 2: Case study cross validation

(see Exercise 5.4.5)

In this case study we use the credit card dataset to predict the probability of default. We will build a logistic regression model and estimate its test error using the validation set approach and the cross-validation approach.

In [24]:
# run this cell to load the data
Default = load_data('Default')
Default

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


In [25]:
Default.dtypes

default     object
student     object
balance    float64
income     float64
dtype: object

Background information on the dataset can be found [in the documentation](https://islp.readthedocs.io/en/latest/datasets/Default.html).

## Task 2.1
Fit a logistic regression model that uses `income` and `balance` to predict `default`.

In [26]:
predictors = Default.columns.drop(['default','student'])
design = MS(predictors).fit(Default)
X = design.transform(Default)
y = Default.default.map(
    {'No': 0,
    'Yes': 1}
)

model = sm.Logit(y,X)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Tue, 04 Jun 2024   Pseudo R-squ.:                  0.4594
Time:                        14:10:26   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    -11.5405      0.435    -26.544      0.000     -12.393     -10.688
balance        0.0056      0.000     24.835      0.000       0.005       0.006
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

## Task 2.2
Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

i. Split the sample set into a training set and a validation set.

ii. Fit a multiple logistic regression model using only the training observations.

iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.

iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [27]:
# i. splitting into validation and test set
train, val = train_test_split(
    Default,
    test_size=0.3,
    random_state = 42
)

In [28]:
# ii. fit logistic regression model using only training observations
predictors = train.columns.drop(['default','student'])
design2 = MS(predictors).fit(train)
X2 = design2.transform(train)
y2 = train.default.map(
    {'No': 0,
    'Yes': 1}
)

model2 = sm.Logit(y2,X2)
results2 = model2.fit()
results2.summary()

Optimization terminated successfully.
         Current function value: 0.078256
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                 7000
Model:                          Logit   Df Residuals:                     6997
Method:                           MLE   Df Model:                            2
Date:                Tue, 04 Jun 2024   Pseudo R-squ.:                  0.4743
Time:                        14:10:26   Log-Likelihood:                -547.79
converged:                       True   LL-Null:                       -1042.0
Covariance Type:            nonrobust   LLR p-value:                2.273e-215
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    -11.6019      0.522    -22.236      0.000     -12.624     -10.579
balance        0.0057      0.000     20.976      0.000       0.005       0.006
income      1.872e-05   5.92e-06      3.163      0.002    7.12e-06    3.03e-05
==============================================================================

Possibly complete quasi-separation: A fraction 0.15 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [29]:
# iii. Prediction for validation set
X_val = design2.transform(val)
predicted_probs = results2.predict(X_val)
predicted_labels = np.where(predicted_probs > 0.5, 'Yes', 'No')

In [30]:
# iv. validation set error
true_labels = val.default
val_err = 1 - np.mean(predicted_labels == true_labels)
val_err

0.026666666666666616

Thus, the test error estimated by the validation set method is $2.7\%$.
Note that this measure overestimates the quality of the model. While only $2.7\%$ of misclassified observations sounds like a good model, the cell below shows that the model produces quite a high number of false negatives (i.e. defaults which are predicted as non-defaults by the model).

In [31]:
predicted_labels[true_labels == 'Yes']

array(['No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes',
       'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'No',
       'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes',
       'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No',
       'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No',
       'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No',
       'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No',
       'No'], dtype='<U3')

In [32]:
from ISLP import confusion_table
confusion_table(predicted_labels, true_labels)

Truth,No,Yes
Predicted,,
No,2896,70
Yes,10,24


## Task 2.3
Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [33]:
# your code here
val_errors = np.zeros(3)
for i in range(3):
    train, val = train_test_split(
        Default,
        test_size=0.3
    )
    predictors = train.columns.drop(['default','student'])
    design3 = MS(predictors).fit(train)
    X3 = design3.transform(train)
    y3 = train.default.map(
        {'No': 0,
        'Yes': 1}
    )

    model3 = sm.Logit(y3,X3)
    results3 = model3.fit()
    X_val = design3.transform(val)
    predicted_probs = results2.predict(X_val)
    predicted_labels = np.where(predicted_probs > 0.5, 'Yes', 'No')
    true_labels = val.default
    val_err = 1 - np.mean(predicted_labels == true_labels)
    val_errors[i] = val_err
val_errors

Optimization terminated successfully.
         Current function value: 0.082333
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077183
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.076652
         Iterations 10


array([0.023     , 0.029     , 0.02833333])

**Comment**: The fraction of observations which are misclassified varies slightly due to the fact that we use different validation sets (no random seed set!) in each of the three iterations.


## Task 2.4
Now predict the test error of the model using 10-fold cross-validation. To do so, follow the steps we developed in Part 1b) of this notebook.

**Note**: To carry out this task, we need to define our own custom score using [`sklearn.make_scorer()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html) and pass this to the function [`sklearn.cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) by specifying it as an argument to the `scoring` parameter.

Our custom scoring function needs to have the signature `score_func(y, y_pred, **kwargs)` with `y` being the true labels and `y_pred` the predicted labels as output by `sm.Logit()` when applying the `predict()` method. Since `statsmodels` outputs probabilities rather than actual labels, we first transform these probabilities  into labels. This is what our scoring function `accuracy_score_sm()` does.

In [ ]:
# your code here

**No solutions beyond this point as we did not go further in class**

## Task 2.5
Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using 10-fold cross-validation. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [ ]:
# your code here

# Part 3: Implementation of bootstrap

## Estimating the accuracy of a statistic
### Introducing the dataset
We closely follow an example presented in [Computational and Inferential Thinking](https://inferentialthinking.com/chapters/13/3/Confidence_Intervals.html).

In [ ]:
url = 'https://drive.google.com/uc?id='
file_id = "15xUDQPqkzKJBoxrafC9iNz4EgFlwbmM_"
births = pd.read_csv(url + file_id)
births

**Task 1**: Create a new column `Birth Weight (g)` which contains the birth weight in kg. Use the fact 1oz = 28.3495g for your computations.

In [ ]:
...

Birth weight is an important factor in the health of a newborn infant. Smaller babies tend to need more medical care in their first days than larger newborns. It is therefore helpful to have an estimate of birth weight before the baby is born. One way to do this is to examine the relationship between birth weight and the number of gestational days.

A simple measure of this relationship is the ratio of birth weight to the number of gestational days. The table ratios contains the first two columns of baby, as well as a column of the ratios. The first entry in that column was calculated as follows:
$$ \frac{3401.94 \text{oz}}{284 \text{ days}} \approx 11.98 \text{g} \text{ per day}.$$

In [ ]:
ratios = pd.DataFrame({
    'Birth Weight' : births['Birth Weight (g)'],
    'Ratio BW:GD' : births['Birth Weight (g)'] / births['Gestational Days']
})
ratios

**Task 2:** Plot a histogram of the ratios.

In [ ]:
fig,ax  = plt.subplots(figsize =(8,8))
sns.histplot(ratios, x = "Ratio BW:GD");

**Task 3:** Compute the median ratio and the maximum ratio in the sample.

In [ ]:
...

In [ ]:
...

### Bootstrapping for estimating the variability of the population median
We now want to estimate the population median. For this we are going to use the bootstrapping method.
We start by reviewing the idea in a graphical manner:
![bootstrap.png](bootstrap.png)

**Task 1**: Define a function `one_bootstrap_median` which will bootstrap the sample and return the median ratio in the bootstrapped sample.

- To bootstrap the sample use the Pandas.DataFrame method [`sample`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html). *Important*: Make sure to draw a sample of the same length as our original sample and make sure to sample with replacement.
- To compute the appropriate quantile, use the Numpy method [`quantile`](https://numpy.org/doc/stable/reference/generated/numpy.quantile.html)

In [ ]:
...

**Task 2**: Initialize a Numpy vector `bootstrap_medians` with zeros of length 5000 (use the Numpy method [`zeros()`](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html#numpy.zeros). Then fill this vector with 5000 bootstrapped medians.

In [ ]:
...

In [ ]:
fig,ax  = plt.subplots(figsize =(8,8))
sns.histplot(x = bootstrap_medians, bins=25);

# Bootstrapping for estimating the accuracy of a Linear Regression Model
Now, we discuss how to use bootstrapping in order to assess the variability of the coefficient estimates and predictions from a statistical learning method. As an example, we look at a simple linear regression model based on the `Auto` dataset which predicts the `mpg` variable based on `horsepower`.

With the bootstrap method we are going to estimate the distribution of the coefficient for `mpg` in this model and we compare the standard error of this coefficient as estimated by `statsmodels` with our bootstrap estimate.

**Task 1**: Define a function `one_bootstrap_model_coefficient` which creates a single bootstrap sample from the Auto dataframe, computes a regression model based on the single predictor `horsepower` and returns the model coefficient for `horsepower`.

In [ ]:
...

**Task 2**: Initialize a Numpy vector `bootstrap_model_coefficients` with zeros of length 5000 (use the Numpy method [`zeros()`](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html#numpy.zeros). Then fill this vector with 5000 bootstrapped model coefficients.

In [ ]:
...

In [ ]:
fig,ax  = plt.subplots(figsize =(8,8))
sns.histplot(x = bootstrap_model_coefficients, bins=25);

**Task 3**: Estimate the standard error of the model coefficient for `horsepower` and assign it to the variable `standard_error_bootstrap`. Compare to the `sm.OLS()` estimate which should be assigned to the variable `standard_error_bootstrap`.

In [ ]:
...

print('Bootstrapped standard error for model coefficient:', "{:10.4f}".format(standard_error_bootstrap))
print('Statsmodels OLS standard error estimate for model coefficient:', "{:10.4f}".format(standard_error_statsmodels))

# Part 4: Case study bootstrap

We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefficients in two different ways: 
1. using the bootstrap, and 
2. using the standard formula for computing the standard errors in the sm.GLM() function.

## Task 4.1
Using the `summarize()` and `sm.GLM()` functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors.

In [ ]:
# your answer here

## Task 4.2
Following the bootstrap example in Part 3 above, estimate the standard errors of the logistic regression coefficients for income and balance with the bootstrap.

In [ ]:
# your answer here

## Task 4.3
Comment on the estimated standard errors obtained using the `sm.GLM()` function and using the bootstrap.

*your comment here*